In [1]:
import json
import pandas as pd
import gzip
import numpy as np
from sklearn.externals import joblib

from collections import Counter

In [2]:
events = []
# File from: wget http://data.gharchive.org/2018-07-01-15.json.gz
with gzip.open('2018-07-01-15.json.gz') as f:
    for line in f:
        events.append(json.loads(line.strip()))

In [3]:
Counter((event['type'] for event in events));

In [4]:
issues = [event for event in events if event['type'] == 'IssuesEvent']
comments = [event for event in events if event['type'] == 'IssueCommentEvent']
CreateEvents = [event for event in events if event['type'] == 'CreateEvent']

In [5]:
actors=[]
#Hold a list of all the usernames and repo names
#This is needed in order to pull README.md files
names=[]
## pull all of the createEvents in order to get UserID and Repo name for 
# calls to get Readme Content
CreateEvents = [event for event in events if event['type'] == 'CreateEvent']
for event in range(1,len(CreateEvents)):
    thisEvent= CreateEvents[event]
    actors.append(thisEvent['repo'])
    for name in range(0,len(actors)):
        thisName=actors[name]
        names.append(thisName['name'])

In [6]:
## gets rid of doubles:
names = np.array(names)
names = np.unique(names)

In [7]:
from github import Github
# First create a Github instance:
## I added this line with my authentivcation so that we dont have to keep using our login info
## I still have to trouble shoot this but I will do it after the code pulls the readMe files
#Probably some time tomorrow.

g = Github("6efe0a63859ff9f96f2e1b829db3525b33d2f533 ")

## This code will pull the README.md files for all of the repos and usernames:
readMeFiles=[]
for repo in range(0,len(names)):
    try:
        thisRepo=g.get_repo(names[repo])
        readMe=thisRepo.get_contents('README.md')
        readMe = readMe.decoded_content
    except:
        pass
        readMe ="No README.md found."
    readMeFiles.append(readMe)
len(readMeFiles)

3529

In [55]:
forks = []
watchers = []
stargazers = []
subscribers = []
for repoName in range(0,len(names)):
    thisRepo=g.get_repo(names[repoName])
    try:
        forks.append(thisRepo.forks_count)
    except:
        forks.append(0)
    try:
        watchers.append(thisRepo.watchers_count)
    except:
        watchers.append(0)
    try:
        stargazers.append(thisRepo.stargazers_count)
    except:
        stargazers.append(0)
    try:
        subscribers.append(thisRepo.subscribers_count)
    except:
        subscribers.append(0)

In [60]:
## create a pandas dataframe holding all the variables for tracking the 
# 'popularity' measures of the repos:
readMesAndPopularity = pd.DataFrame(
    {'username/repoName': names,
     'forks': forks,
     'stargazers': stargazers,
     'subscribers': subscribers,
     'ReadMes': readMeFiles
    })

In [62]:
joblib.dump(readMesAndPopularity, "readMeFiles.pkl")

['readMeFiles.pkl']

In [61]:
readMesAndPopularity.head()

,username/repoName,forks,stargazers,subscribers,ReadMes
0,0day-ci/linux,11,15,6,No README.md found.
1,0xboobface/ctbrec,0,1,0,b'# CTB Recorder\n\nA free recording software ...
2,100488Dflc/Deep_Learning_Uniandes_DFLC,0,0,0,b'# Deep_Learning_Uniandes_DFLC'
3,100488Dflc/Deep_Learning_Uniandes_DL,0,0,0,No README.md found.
4,1206497655/tree,0,0,0,b'\xef\xbb\xbf\xe5\x89\x8d\xe4\xba\x9b\xe5\xa4...


In [ ]:
readMeFiles = joblib.load("readMeFiles_.pkl")

3